In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

%matplotlib inline
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

import sklearn.model_selection as ms

sns.set(style='white', context='notebook', palette='deep')

# Data preparation

In [ ]:
# Load the data
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")
train.shape, test.shape

**Checking for missing values**

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

*Since, there is no missing data found, we can proceed further.*

**Separating data into features and labels**

In [ ]:
y = train['label'].to_numpy()
train = train.drop(columns=['label'])

In [ ]:
# Getting common numpy values for the dataframe
X = train.to_numpy()
test = test.to_numpy()

*Suppose there are values in the Dataframe or Series having dtypes float16 and float32, then after applying this method, all values will have a common dtype.*

**Reshaping**

In [ ]:
X = X.reshape(-1,28,28,1)
test = test.reshape(-1,28,28,1)

**Normalization**

In [ ]:
X = X/255.0
test = test/255.0

*CNN converg faster on [0..1] data than on [0..255] and it's a lot easier to interpret also.*

In [ ]:
indices = np.random.randint(low=0, high=42000, size=5)
for i in indices:
    plt.figure()
    plt.imshow(X[i,:,:,0], cmap='gray')

**Label Encoding**

In [ ]:
# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
y = to_categorical(y, 10)

**Split training and validation set**

In [ ]:
# Seed is set in order to retain consistent results
np.random.seed(1)

# Train_test_split
X_train, X_val, y_train, y_val = ms.train_test_split(X, y, test_size=0.1, random_state=1)

**Here, we have built a Sequential model with 5 convolution layers with the image size 28X28 and a Dense layer with 512 neurons. We have also specified callbacks based on validation accuracy(patience means process will come to a halt after the no. of epochs for which val_acc doesn't improve).**

In [ ]:
# EarlyStopping and Reduce Learning Rate Callbacks
my_callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=5)
my_callback_rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', patience=2, factor=0.5, min_lr=0.00001, verbose=1)

model = tf.keras.models.Sequential([
    # The input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution layer
    tf.keras.layers.Conv2D(64, (5,5), padding='same', activation='relu', input_shape=(28, 28, 1)),

    # This is the second convolution layer
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu', strides=2),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # This is the third convolution layer
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    
    # This is the fourth convolution layer
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    
    
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.summary()
model.compile(optimizer=RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[my_callback_es, my_callback_rlr], shuffle=True, batch_size=86)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.ylim(top=1.0)
plt.title('Training and validation loss')
plt.legend(loc=0)

plt.show()

In [ ]:
predictions = model.predict(test)
predictions = predictions.argmax(axis=1)
imageid = np.arange(1,28001)
output = pd.DataFrame({"ImageId": imageid,"Label": predictions})
output.to_csv('submissions1.csv', index=False)